---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import re

In [2]:
def blight_model():
    
    #print("---> 1. read_data and exclude complicance is null and feature that's not in test set")

    data_train = pd.read_csv("train.csv", encoding = 'ISO-8859-1' , low_memory=False)
    data_test = pd.read_csv("test.csv", encoding = 'utf-8-sig')
    data_address = pd.read_csv("addresses.csv", encoding = 'utf-8-sig')
    data_latlons = pd.read_csv("latlons.csv", encoding = 'utf-8-sig')
    data_address_all = pd.merge(data_address, data_latlons, on = "address", how = 'inner')

    # check ticket_id in data_address_all for train and test data
    data_train[~data_train["ticket_id"].isin(data_address_all["ticket_id"])].shape
    data_test[~data_test["ticket_id"].isin(data_address_all["ticket_id"])].shape

    data_train = pd.merge(data_train, data_address_all, on = 'ticket_id', how = 'left')
    data_test = pd.merge(data_test, data_address_all, on = 'ticket_id', how = 'left')

    # exclude compliance is null
    #print("# of na in target:", data_train[pd.isnull(data_train["compliance"])].shape[0])
    data_train = data_train[pd.notnull(data_train["compliance"])]


    # exclude feature that's not in test set as well as some useless feature
    # useless: 'inspector_name', 'violator_name', 'admin_fee', 'state_fee', 'late_fee', 'clean_up_cost'
    #           'judgment_amount' -> correlation with fine_amount, 'grafitti_status' -> na
    #           'violation_street_name' -> same as address

    # note: check 'hearing_date' and 'ticket_issued_date' useful or not
    col_keep = ['ticket_id', 'agency_name', \
                'city', 'state',\
               'violation_code', 'disposition', 'fine_amount',\
               'discount_amount', 'address']

    data_test = data_test[col_keep]       
    #data_train_target = data_train[["compliance"]]
    col_keep.append("compliance")
    data_train = data_train[col_keep]

    # variable used: agency_name, top 5 city, top 5 states, top 5 violation code, disposition, fine_amount, discount_amount
    #                top 5 violation street


    # top 5 city
    # change the city name to upper case since there are mix in the data
    data_train["city"] = data_train["city"].apply(lambda row: str(row).upper().strip())
    data_test["city"] = data_test["city"].apply(lambda row: str(row).upper().strip())


    # get top 5 ticket issued state
    top_5_high_ticket_state = data_train.groupby(by=["state"])["ticket_id"] \
                            .size().reset_index() \
                            .sort_values(by=0, ascending = False)[:5]["state"].tolist()

    data_train_top5_state = data_train[data_train["state"].isin(top_5_high_ticket_state)]    

    # get top 5 violator living city
    top_5_living_city = data_train.groupby(by=["city"])["ticket_id"] \
                            .size().reset_index() \
                            .sort_values(by=0, ascending = False)[:5]["city"].tolist()
    data_train_top5_living_city = data_train[data_train["city"].isin(top_5_living_city)]  



    # get top 5 violation code
    top_5_violation_code = data_train.groupby(by=["violation_code"])["ticket_id"] \
                            .size().reset_index() \
                            .sort_values(by=0, ascending = False)[:5]["violation_code"].tolist()
    data_train_top5_violation = data_train[data_train["violation_code"].isin(top_5_violation_code)]  

    # top 5 violation street
    # extract part of the violation address
    data_train["violation_street"] = data_train["address"].apply(lambda row: re.split(",", row)[0].strip())
    data_train["violation_street"] = data_train["violation_street"].apply(lambda row: re.findall("[A-Za-z].*", row.strip())[0])          
    data_test["violation_street"] = data_test["address"].apply(lambda row: re.split(",", row)[0].strip())
    data_test["violation_street"] = data_test["violation_street"].apply(lambda row: re.findall("[A-Za-z].*", row.strip())[0])          


    # get top 5 road
    top_5_violation_street = data_train.groupby(by=["violation_street"])["ticket_id"] \
                            .size().reset_index() \
                            .sort_values(by=0, ascending = False)[:5]["violation_street"].tolist()
    data_train_top5_vio_str = data_train[data_train["violation_street"].isin(top_5_violation_street)]  


    # change variable for discount_amount

    # if discount_amount > 0 then 1 else 0
    data_train["discount_amount"] = data_train["discount_amount"].apply(lambda row: 1 if row>0 else 0)
    data_test["discount_amount"] = data_test["discount_amount"].apply(lambda row: 1 if row>0 else 0)
    #data_train.groupby(by=["discount_amount","compliance"])["ticket_id"].size()    

    #print("---> 2. onehotencode the feature to both data_train & data_test")

    full = [data_train, data_test]
    for df in full:
        for i in range(len(top_5_high_ticket_state)):
            df["state" + "_" + top_5_high_ticket_state[i]] = 0
            df["state" + "_" + top_5_high_ticket_state[i]] = np.where(df['state'].str.contains(top_5_high_ticket_state[i]), 1, 0)    
    
    for df in full:
        for i in range(len(top_5_living_city)):
            df["city" + "_" + top_5_living_city[i]] = 0
            df["city" + "_" + top_5_living_city[i]] = np.where(df['city'].str.contains(top_5_living_city[i]), 1, 0)    
    
    for df in full:
        for i in range(len(top_5_violation_code)):
            df["violation_code" + "_" + top_5_violation_code[i]] = 0
            df["violation_code" + "_" + top_5_violation_code[i]] = np.where(df['violation_code'].str.contains(top_5_violation_code[i]), 1, 0)    
    
    for df in full:
        for i in range(len(top_5_violation_street)):
            df["violation_street" + "_" + top_5_violation_street[i]] = 0
            df["violation_street" + "_" + top_5_violation_street[i]] = np.where(df['violation_street'].str.contains(top_5_violation_street[i]), 1, 0)    
    
    
    list_agency_name = pd.unique(data_train['agency_name']).tolist()

    for df in full:
        for i in range(len(list_agency_name)):
            df["agency" + "_" + list_agency_name[i]] = 0
            df["agency" + "_" + list_agency_name[i]] = np.where(df['agency_name'].str.contains(list_agency_name[i]), 1, 0)    
    
    
    list_disposition = pd.unique(data_train['disposition']).tolist()

    for df in full:
        for i in range(len(list_disposition)):
            df["disposition" + "_" + list_disposition[i]] = 0
            df["disposition" + "_" + list_disposition[i]] = np.where(df['disposition'].str.contains(list_disposition[i]), 1, 0)    
    
    
    for df in full:  
        del df["city"], df["state"], df["violation_code"], df["agency_name"], df["disposition"], df["address"], df["violation_street"]
    
    
    #print("---> 3. feature scaling")
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    y_train = data_train["compliance"]
    col_keep_1 = data_train.columns.tolist()
    X_train = data_train[[col for col in col_keep_1 if col not in ["compliance"]]]

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)


    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    # we must apply the scaling to the test set that we computed for the training set
    X_test_scaled = scaler.transform(X_test)
    test_scaled = scaler.transform(data_test)    
    
    
    # random forest
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    clf = RandomForestClassifier(class_weight = 'balanced', n_estimators=100, max_depth=3, random_state=0).fit(X_train_scaled, y_train)
    clf_pred = clf.predict(X_test_scaled)
    #print("random forest accuracy:", clf.score(X_train_scaled, y_train))
    #print("cross_val_score:", np.mean(cross_val_score(clf, X_train_scaled, y_train , cv=10)))

    
    # first item is probability of being 0 and second column is being 1  
    
    return pd.Series(clf.predict_proba(test_scaled)[:, 1] , index=data_test["ticket_id"])

In [3]:
blight_model()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:97: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:118: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


ticket_id
284932    0.455309
285362    0.386560
285361    0.455309
285338    0.452748
285346    0.461604
285345    0.452748
285347    0.461604
285342    0.715217
285530    0.369061
284989    0.389061
285344    0.463867
285343    0.400855
285340    0.386560
285341    0.454918
285349    0.458934
285348    0.450078
284991    0.395005
285532    0.389908
285406    0.389908
285001    0.363058
285006    0.363058
285405    0.383718
285337    0.374404
285496    0.461214
285497    0.452748
285378    0.383718
285589    0.395005
285585    0.445587
285501    0.455309
285581    0.386560
            ...   
376367    0.358012
376366    0.363505
376362    0.669016
376363    0.683099
376365    0.358012
376364    0.363505
376228    0.363505
376265    0.368551
376286    0.668378
376320    0.363505
376314    0.363505
376327    0.682895
376385    0.682895
376435    0.650761
376370    0.668712
376434    0.448528
376459    0.448528
376478    0.384910
376473    0.368551
376484    0.380212
376482    0.361023
37